In [1]:
import numpy as np
import magpylib as magpy
from magpylib.magnet import Cuboid, Cylinder, CylinderSegment
from maggeometry import n_rings
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

### Nelder-Mead Adaptive Optimization of 3 rings

### Pippin 1

All variables can be optimized independently for each ring set

1. Run 50 shots using Nelder-Mead with nonuniformity cost function
2. Optimize for field strength using new cost function

In [2]:
Br = 1.09e3
mag_dir = (0,0,1)
mirror1_z = False
ringset_1_config = [Br, mag_dir, mirror1_z]

r = 1.09e3
mag_dir = (0,0,1)
mirror2_z = True
ringset_2_config = [Br, mag_dir, mirror2_z]
# innerrad, width, thickness, dist
bounds = ((1, 150), (2, 20), (2, 20),
          (1, 150), (2, 20), (2, 20), (1, 300))

In [3]:
from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
def obj1_nonuniformity(x):
    obj_ring_config=[[*ringset_1_config, *x[0:3], 0], [*ringset_2_config, *x[3:7]]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    return av_nonuniformity

In [4]:
import numpy as np
from scipy.optimize import minimize, Bounds
from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid
from alive_progress import alive_bar

options = {'disp':True, 'fatol': 1e-12, 'maxiter':5e3, 'adaptive':True}
method = "Nelder-Mead"

shots = 50
results_x = []
results_fun = []
guesses_x0 = []
with alive_bar(shots, force_tty=True) as bar:
    for i in range(shots):
        x0 = np.zeros(len(bounds))
        for b in range(len(bounds)):
            x0[b] = np.random.uniform(low=bounds[b][0], high=bounds[b][1])
        res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)
        guesses_x0.append(x0)
        results_x.append(res.x)
        results_fun.append(res.fun)
        bar()

print("Objective function values:")
print(results_fun)
print("Result parameters:")
print(results_x)
print("Starting guesses:")
print(guesses_x0)

on 0: /var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_57113/2223040715.py:19: RuntimeWarning: Maximum number of iterations has been exceeded.
        res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)


on 2: Optimization terminated successfully.                                     
on 2:          Current function value: 0.006003                                 
on 2:          Iterations: 1230                                                 
on 2:          Function evaluations: 2011                                       
on 3: Optimization terminated successfully.                                     
on 3:          Current function value: 0.001535                                 
on 3:          Iterations: 2876                                                 
on 3:          Function evaluations: 4669                                       
on 4: Optimization terminated successfully.                                     
on 4:          Current function value: 0.006003                                 
on 4:          Iterations: 1179                                                 
on 4:          Function evaluations: 1934                                       
on 5: Optimization terminate

In [9]:
from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid, get_grid_mag_and_nonuniformity
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results_g_center = []
for x in results_x:
    obj_ring_config=[[*ringset_1_config, *x[0:3], 0], [*ringset_2_config, *x[3:7]]]
    magnets = n_rings(obj_ring_config)
    Gmag, Gnon, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_mag_and_nonuniformity(magnets, grid, grid_res, True)
    results_g_center.append(center_field)
print(results_g_center)

[-56.70580950826874, -225.86983621418062, -84.98071648161829, -5420.292930702744, -84.98071474225718, -84.74499612485467, -84.98071707137484, -84.98071711177775, -84.98071711177775, -8.54205557392733, 24.946678618662716, -84.9807160085458, -49.59692927116876, -90.34203188794272, -9.26246962513698, -84.98071299557864, -8.35468154908086, -84.980715972746, -5349.781552739082, -168.66863243239465, -5428.995306643001, -84.98071711177775, -74.60513658009779, -84.74498962068654, -84.98071629328382, -84.98068864071456, -84.98070165531821, -32.18473673123686, -84.98070933915628, -84.98071592377684, -8.542055331889822, -84.98071529594739, -84.98071711177775, -63.86820328911661, -84.9806852131178, -84.56635708415249, -24.764137419016876, -84.98071322234281, -49.58522401392477, 18.396922889306033, -84.98071626842417, -21.423074089744745, 27.022225519785152, -11.189292536769386, -84.98071615545932, -8.223830110404728, -38.17337726450127, -102.0811713511953, -47.60727072371466, 28.152875643972585]


In [10]:
from maghelper import make_opt_res_csv
import datetime
today = datetime.datetime.today().strftime('%Y_%m_%d')
# insert dist1 = 0 column
r_x = np.array(results_x)
r_x = np.insert(r_x, 3, 0, axis=1)
r_g = np.array(guesses_x0)
r_g = np.insert(r_g, 3, 0, axis=1)

b, b_df = make_opt_res_csv(f'results/3 rings/pippin/run1/{today}_pippin1_3r_1090mT_', results_fun, results_g_center, r_x, r_g)

Number of unique results: 49
    nonuniformity  center_field_gauss  r_1_innerrad  r_1_width  r_1_thickness  \
0    4.373026e-08          -24.764137    111.880297   3.732331       5.007847   
1    4.601974e-08          -11.189293    134.203719   2.278618      12.060322   
2    4.715034e-08          -38.173377    102.382532   2.084809       6.761673   
3    8.335235e-08          -49.596929    150.000000  14.595483      15.641595   
4    9.229134e-08          -84.566357    106.615741  13.044977       8.659620   
5    1.217685e-07          -32.184737    137.966817  15.754542       8.038976   
6    1.248944e-07           -9.262470    150.000000  15.496415       2.818520   
7    1.364210e-07          -56.705810    103.719162   6.067344      11.626395   
8    1.404622e-07           -8.223830    149.945273  20.000000       2.000000   
9    1.679902e-07           -8.354682    150.000000  18.558711       2.178223   
10   1.735968e-07          -90.342032    112.986887  15.038231      14.323499   

### Confirmation of field direction

In [19]:
import numpy as np
import pandas as pd


stest = pd.read_csv('results/3 rings/pippin/run1/2023_08_01_pippin1_3r_1090mT_49.csv')
stest = stest.to_numpy()
stestu = stest[:, [2,3,4,5,6,7,8,9,10]]
# stestu = np.unique(stestu, axis=0)
_, idx = np.unique(stestu, axis=0, return_index=True)
stestu = stestu[np.sort(idx)]
nonmatch = []
corrected_g_center = []
corrected_nonun = []
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)

Br = 1.09e3
mag_dir = (0,0,1)
mirror1_z = False
ringset_1_config = [Br, mag_dir, mirror1_z]

r = 1.09e3
mag_dir = (0,0,1)
mirror2_z = True
ringset_2_config = [Br, mag_dir, mirror2_z]

with alive_bar(len(stestu), force_tty=True) as bar:
    for i in range(len(stestu)):
        s0 = stestu[i]
        obj_ring_config=[[*ringset_1_config, *s0[1:5]], [*ringset_2_config, *s0[5:9]]]
        magnets = n_rings(obj_ring_config)
        grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res, True)
        grid_field_2, center_field_mag, av_nonuniformity_2, max_abs_nonuniformity_2 = get_grid_nonuniformity(magnets, grid, grid_res, False)
        if (round(center_field, 12) == round(s0[0], 12)):
            print(i, "matches")
        else:
            print(i, "does not match", round(center_field, 12), round(s0[0], 12))
            nonmatch.append(i)
        corrected_g_center.append(center_field)
        corrected_nonun.append(av_nonuniformity)
        bar()
if (len(nonmatch) > 0):
    print(nonmatch)
else:
    print("All field directions are parallel with magnetization direction")

on 0: 0 matches                                                                 
on 1: 1 matches                                                                 
on 2: 2 matches                                                                 
on 3: 3 matches                                                                 
on 4: 4 matches                                                                 
on 5: 5 does not match -32.184736731235 -32.184736731237                        
on 6: 6 matches                                                                 
on 7: 7 matches                                                                 
on 8: 8 matches                                                                 
on 9: 9 does not match -8.354681549082 -8.354681549081                          
on 10: 10 matches                                                               
on 11: 11 matches                                                               
on 12: 12 matches           

### Pippin stage 2

$C = \eta \times ||B_0| - 10|$

Change in absolute value usage may allow convergence to -10G.

Use only unique results to reduce redundant calculation.

To run on 8/1/2023

In [31]:
import numpy as np
import pandas as pd

st1 = pd.read_csv('results/3 rings/pippin/run1/2023_08_01_pippin1_3r_1090mT_49.csv')
st1 = np.delete(st1.to_numpy(), 0, axis=1) #remove first column of indices
print (st1)

[[ 4.37302580e-08 -2.47641374e+01  1.11880297e+02  3.73233088e+00
   5.00784735e+00  0.00000000e+00  1.49378173e+02  5.59458748e+00
   1.99194144e+01  7.30985968e+01  5.86794081e+01  5.47581190e+00
   1.49475008e+01  0.00000000e+00  1.46989337e+02  4.69247178e+00
   1.90777367e+01  6.58801416e+01]
 [ 4.60197373e-08 -1.11892925e+01  1.34203719e+02  2.27861823e+00
   1.20603215e+01  0.00000000e+00  1.29583926e+02  5.05016717e+00
   6.45956664e+00  7.74926332e+01  3.88061921e+01  4.69350109e+00
   1.86557560e+01  0.00000000e+00  1.17633854e+02  4.23105863e+00
   5.54545969e+00  5.40996483e+01]
 [ 4.71503421e-08 -3.81733773e+01  1.02382532e+02  2.08480922e+00
   6.76167290e+00  0.00000000e+00  1.49986046e+02  2.00000000e+01
   9.40328333e+00  7.08629295e+01  1.19253413e+02  1.18792965e+01
   1.00142463e+01  0.00000000e+00  4.72389979e+01  1.78331906e+01
   1.33840912e+01  1.11091631e+01]
 [ 8.33523522e-08 -4.95969293e+01  1.50000000e+02  1.45954827e+01
   1.56415953e+01  0.00000000e+00  9.

In [32]:
Br = 1.09e3
mag_dir = (0,0,1)
mirror1_z = False
ringset_1_config = [Br, mag_dir, mirror1_z]

mag_dir = (0,0,1)
mirror2_z = True
ringset_2_config = [Br, mag_dir, mirror2_z]
# innerrad, width, thickness, dist
bounds2c = ((1, 150), (2, 20), (2, 20), (1, 150), (2, 20), (2, 20), (1, 300))

### Batch 1 run with 25 completed Pippin 2

In [33]:
from maggeometry import n_rings, three_rings
# 10G target
field_strength_target = 10
dist1 = 0

grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
def product_nonuniformity(x):
    innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2 = x
    obj_ring_config=[[*ringset_1_config, *x[0:3], 0], [*ringset_2_config, *x[3:7]]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = av_nonuniformity*np.abs(np.abs(center_field) - field_strength_target)
    return cost

In [37]:
results2c_x = []
results2c_fun = []
method2c = 'Nelder-Mead'
options2c = {'disp': True, 'fatol': 1e-18, 'maxiter': 5e3, 'adaptive': True}
with alive_bar(len(st1), force_tty=True) as bar:
    for s0 in st1:
        res2c = minimize(product_nonuniformity, [*s0[2:5], *s0[6:10]], method=method2c, options=options2c, bounds=bounds2c)
        results2c_x.append(res2c.x)
        results2c_fun.append(res2c.fun)
        bar()
        
print(results2c_x)
print(results2c_fun)

on 0: /var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_57113/1500555619.py:7: RuntimeWarning: Maximum number of iterations has been exceeded.
        res2c = minimize(product_nonuniformity, [*s0[2:5], *s0[6:10]], method=method2c, options=options2c, bounds=bounds2c)


on 1: Optimization terminated successfully.                                     
on 1:          Current function value: 0.000000                                 
on 1:          Iterations: 1456                                                 
on 1:          Function evaluations: 2483                                       
on 6: Optimization terminated successfully.                                     
on 6:          Current function value: 0.000000                                 
on 6:          Iterations: 929                                                  
on 6:          Function evaluations: 1650                                       
on 9: Optimization terminated successfully.                                     
on 9:          Current function value: 0.000000                                 
on 9:          Iterations: 2068                                                 
on 9:          Function evaluations: 3433                                       
on 16: Optimization terminat

KeyboardInterrupt: 

In [38]:
from maggeometry import n_rings, three_rings
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results2c_nonun = []
results2c_g_center = []
for x in results2c_x:
#     innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2 = x
    obj_ring_config=[[*ringset_1_config, *x[0:3], 0], [*ringset_2_config, *x[3:7]]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res, True)
    results2c_nonun.append(av_nonuniformity)
    results2c_g_center.append(center_field)

print(results2c_nonun)
print(results2c_g_center)

[7.648727135317644e-08, 8.988435052817357e-08, 4.7202797446527825e-08, 8.335231255919888e-08, 1.1656134949171528e-07, 8.701407626153737e-08, 2.7726476338535007e-07, 1.305964960644152e-07, 1.4046627994444082e-07, 4.0459218598615873e-07, 8.90345155544971e-08, 1.8077054167610495e-07, 2.785559236039148e-07, 2.869649758950451e-07, 4.246004164142331e-07, 3.5469473255188275e-07, 1.1278044138455272e-05, 7.684663407081744e-06, 9.231063808108664e-06, 6.769823922476001e-07, 3.8716126073250465e-05, 1.829769069491812e-06, 1.7961860631102469, 1.7085675816325026, 0.006003319029187648]
[-13.792063259197889, -10.000000000019895, -37.03431692530202, -49.59687012476266, -35.98369536085741, -15.920946939248246, -9.999999999994955, -29.951895164339533, -8.223981968839436, -10.000000000001243, -95.0085019313223, -75.22206198054322, -44.95809325794191, -20.398115894881954, -12.258090493264717, -55.65765037608402, 10.000000000000577, 10.000000000000027, 10.000000000000018, -102.68267169526382, 10.0, -183.8766

In [41]:
from maghelper import make_opt_res2_csv
import datetime
today = datetime.datetime.today().strftime('%Y_%m_%d')
r2c_x = np.array(results2c_x)
r2c_x = np.insert(r2c_x, 3, 0, axis=1)
make_opt_res2_csv(f'results/3 rings/pippin/run1/{today}_pippin2_3r_1090mT_part1_', results2c_nonun, results2c_g_center, r2c_x, results2c_fun)

Number of unique results: 25
    nonuniformity  center_field_gauss  r_1_innerrad  r_1_width  r_1_thickness  \
0    4.720280e-08          -37.034317    102.421875   2.000000       6.848174   
1    7.648727e-08          -13.792063    100.862673   2.000000       3.734003   
2    8.335231e-08          -49.596870    150.000000  14.595475      15.641578   
3    8.701408e-08          -15.920947    149.999999   8.931122       7.893151   
4    8.903452e-08          -95.008502    127.953721  19.090693      15.887582   
5    8.988435e-08          -10.000000    134.364745   2.410195       9.801769   
6    1.165613e-07          -35.983695    102.766745   4.375771       8.345703   
7    1.305965e-07          -29.951895    104.150482   3.899839       8.926003   
8    1.404663e-07           -8.223982    149.945361  20.000000       2.000011   
9    1.807705e-07          -75.222062    142.855721  19.511421      17.021964   
10   2.772648e-07          -10.000000    150.000000  16.245359       2.921785   

(array([[ 4.72027974e-08, -3.70343169e+01,  1.02421875e+02,
          2.00000004e+00,  6.84817369e+00,  0.00000000e+00,
          1.49943509e+02,  2.00000000e+01,  9.11719393e+00,
          7.08516745e+01,  1.27609539e-06],
        [ 7.64872714e-08, -1.37920633e+01,  1.00862673e+02,
          2.00000011e+00,  3.73400250e+00,  0.00000000e+00,
          1.38920590e+02,  2.87737509e+00,  1.83936421e+01,
          6.62389643e+01,  2.90044571e-07],
        [ 8.33523126e-08, -4.95968701e+01,  1.50000000e+02,
          1.45954751e+01,  1.56415776e+01,  0.00000000e+00,
          9.27178413e+01,  1.45278927e+01,  3.41230602e+00,
          7.16038177e+01,  3.30049070e-06],
        [ 8.70140763e-08, -1.59209469e+01,  1.49999999e+02,
          8.93112163e+00,  7.89315087e+00,  0.00000000e+00,
          9.81756247e+01,  2.49518727e+00,  6.52655137e+00,
          7.13381972e+01,  5.15205729e-07],
        [ 8.90345156e-08, -9.50085019e+01,  1.27953721e+02,
          1.90906933e+01,  1.58875818e+01,  

### Batch 2 run with 25 completed Pippin 2

In [44]:
print(st1[25:])

[[ 6.00331903e-03 -8.49807171e+01  1.50000000e+02  2.00000000e+01
   2.00000000e+01  0.00000000e+00  1.00000000e+00  2.00000000e+00
   2.00000000e+00  3.00000000e+02  5.08266828e+01  1.96460006e+01
   1.83737022e+01  0.00000000e+00  1.09280727e+02  8.97802748e+00
   5.90525871e+00  2.62297217e+02]
 [ 6.00331903e-03 -8.49807171e+01  1.50000000e+02  2.00000000e+01
   2.00000000e+01  0.00000000e+00  1.00000000e+00  2.00000000e+00
   2.00000000e+00  3.00000000e+02  1.36931949e+01  7.94827638e+00
   3.79195974e+00  0.00000000e+00  1.24005359e+02  8.75458391e+00
   6.13374880e+00  2.26005059e+02]
 [ 6.00331903e-03 -8.49807171e+01  1.50000000e+02  2.00000000e+01
   2.00000000e+01  0.00000000e+00  1.00000000e+00  2.00000000e+00
   2.00000000e+00  3.00000000e+02  1.27671145e+02  9.98655508e+00
   1.22255928e+01  0.00000000e+00  5.40307937e+01  1.70395932e+01
   9.68128510e+00  1.76689151e+02]
 [ 6.00331903e-03 -8.49807159e+01  1.50000000e+02  2.00000000e+01
   1.99999997e+01  0.00000000e+00  1.

In [42]:
results2c_x = []
results2c_fun = []
method2c = 'Nelder-Mead'
options2c = {'disp': True, 'fatol': 1e-18, 'maxiter': 5e3, 'adaptive': True}
with alive_bar(len(st1[25:]), force_tty=True) as bar:
    for s0 in st1[25:]:
        res2c = minimize(product_nonuniformity, [*s0[2:5], *s0[6:10]], method=method2c, options=options2c, bounds=bounds2c)
        results2c_x.append(res2c.x)
        results2c_fun.append(res2c.fun)
        bar()
        
print(results2c_x)
print(results2c_fun)

on 0: /var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_57113/2632488517.py:7: RuntimeWarning: Maximum number of iterations has been exceeded.
        res2c = minimize(product_nonuniformity, [*s0[2:5], *s0[6:10]], method=method2c, options=options2c, bounds=bounds2c)


|███▎⚠︎                                   | (!) 2/25 [8%] in 50:00.7 (0.00/s)    


KeyboardInterrupt: 

In [74]:
import pandas as pd

guess_threshold = 1.4 # only use results that are within +40% of minimum homogeneity for stage 2

st1 = pd.read_csv('results/3 rings/pippin/run1/2023_07_12_pippin1_3r_1090mT_40.csv')
st1 = np.delete(st1.to_numpy(), 0, axis=1) #remove first column of indices
min_nonun = st1[0][0]
nonun_threshold = min_nonun*guess_threshold
print("Nonuniformity threshold:", nonun_threshold)
st2g = st1[st1[:,0] < nonun_threshold]
print(len(st2g))

Nonuniformity threshold: 2.546849071771814e-08
5


In [95]:
Br = 1.09e3
mag_dir = (0,0,1)
mirror1_z = False
ringset_1_config = [Br, mag_dir, mirror1_z]

r = 1.09e3
mag_dir = (0,0,1)
mirror2_z = True
ringset_2_config = [Br, mag_dir, mirror2_z]
# innerrad, width, thickness, dist
bounds2 = ((1, 150), (2, 20), (2, 20), (1, 150), (2, 20), (2, 20), (1, 300))

In [96]:
from maggeometry import n_rings, three_rings
# 10G target
field_strength_target = 10
dist1 = 0

grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
def obj2_nonuniformity(x):
    innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2 = x
    obj_ring_config = [[Br, mag_dir, mirror1_z, innerrad1, width1, thickness1, dist1],
                       [Br, mag_dir, mirror2_z, innerrad2, width2, thickness2, dist2]]
#     magnets = three_rings(Br, innerrad1, innerrad2, width1, thickness1, dist2)
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = av_nonuniformity*abs(center_field - field_strength_target)
#     cost = np.log10(av_nonuniformity) + abs(center_field - field_strength_target)
    return cost

### Optimize 3 rings 2nd stage

In [101]:
import random
results2_x = []
results2_fun = []
method2 = 'Nelder-Mead'
options2 = {'disp':True, 'fatol': 1e-17,
#             'maxiter':0.5e3
           }
s = 0.05
with alive_bar(len(st2g), force_tty=True) as bar:
    for s0 in st2g:
        innerrad1, width1, thickness1 = s0[2:5]
        innerrad2, width2, thickness2, dist2 = s0[6:10]
        res2 = minimize(obj2_nonuniformity, [innerrad1 + random.uniform(-s, s),
                                             width1 + random.uniform(-s, s),
                                             thickness1 + random.uniform(-s, s),
                                             innerrad2 + random.uniform(-s, s),
                                             width2 + random.uniform(-s, s),
                                             thickness2 + random.uniform(-s, s),
                                             dist2 + random.uniform(-s, s)], method=method2, options=options2, bounds=bounds2)
        print(res2.x)
        results2_x.append(res2.x)
        results2_fun.append(res2.fun)
        bar()
        
print(results2_x)
print(results2_fun)

on 0: Optimization terminated successfully.                                     
on 0:          Current function value: 0.000001                                 
on 0:          Iterations: 371                                                  
on 0:          Function evaluations: 825                                        
on 0: [149.96408028  14.28091921  10.39643967 149.96713395   9.39505863         
        19.93181024  90.26675985]
on 1: Optimization terminated successfully.                                     
on 1:          Current function value: 0.000000                                 
on 1:          Iterations: 258                                                  
on 1:          Function evaluations: 519                                        
on 1: [149.98859337  18.54191095   2.01866164 141.39565166  19.56852278         
         2.00150346  93.40433963]
on 2: [149.99610297   2.0024549    2.00262794 150.           2.14782666         
         2.49365437  87.11728985]


on 2: /var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_50927/587022898.py:13: OptimizeWarning: Initial guess is not within the specified bounds
        res2 = minimize(obj2_nonuniformity, [innerrad1 + random.uniform(-s, s),
/var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_50927/587022898.py:13: RuntimeWarning: Maximum number of function evaluations has been exceeded.
        res2 = minimize(obj2_nonuniformity, [innerrad1 + random.uniform(-s, s),


on 3: [150.          19.96646514  19.99343041 131.41763054  18.41905982         
        16.57609768  87.5692684 ]
on 4: Optimization terminated successfully.                                     
on 4:          Current function value: 0.000000                                 
on 4:          Iterations: 303                                                  
on 4:          Function evaluations: 575                                        
on 4: [149.98316964  19.99412146   2.15249774 133.69494823  17.47981255         
         2.01068101  94.97806183]
|████████████████████████████████████████| 5/5 [100%] in 2:00.8 (0.04/s)        
[array([149.96408028,  14.28091921,  10.39643967, 149.96713395,
         9.39505863,  19.93181024,  90.26675985]), array([149.98859337,  18.54191095,   2.01866164, 141.39565166,
        19.56852278,   2.00150346,  93.40433963]), array([149.99610297,   2.0024549 ,   2.00262794, 150.        ,
         2.14782666,   2.49365437,  87.11728985]), array([150.        ,  

In [102]:
from maggeometry import n_rings, three_rings
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results2_nonun = []
results2_g_center = []
for x in results2_x:
#     innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2 = x
    obj_ring_config=[[*ringset_1_config, *x[0:3], 0], [*ringset_2_config, *x[3:7]]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    results2_nonun.append(obj1_nonuniformity(x))
    results2_g_center.append(center_field)

print(results2_nonun)
print(results2_g_center)

[1.819612755249618e-08, 1.464730496292909e-06, 2.2165801185046723e-08, 2.421328831128445e-08, 3.3461796554650107e-06]
[46.02882371927372, 9.999999999996527, 1.376560421919768, 101.46870197942754, 10.000000000000231]


In [86]:
st2g2 = st1[5:10]
results2_x = []
results2_fun = []
method2 = 'Nelder-Mead'
options2 = {'disp':True, 'fatol': 1e-17,
#             'maxiter':0.5e3
           }
with alive_bar(len(st2g2), force_tty=True) as bar:
    for s0 in st2g2:
        innerrad1, width1, thickness1 = s0[2:5]
        innerrad2, width2, thickness2, dist2 = s0[6:10]
        res2 = minimize(obj2_nonuniformity, [innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2], method=method2, options=options2, bounds=bounds2)
        print(res2.x)
        results2_x.append(res2.x)
        results2_fun.append(res2.fun)
        bar()
        
print(results2_x)
print(results2_fun)

on 0: [120.99652309  11.44496932   4.06953278 147.91960699  15.1965081          
        11.2103605   78.38435151]


on 0: /var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_50927/2344889787.py:12: RuntimeWarning: Maximum number of function evaluations has been exceeded.
        res2 = minimize(obj2_nonuniformity, [innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2], method=method2, options=options2, bounds=bounds2)


on 1: [141.91532763   4.47989082   6.36514709 134.28098985   7.92708789         
         3.97134997  81.85782279]
on 2: [125.30070792  13.38018151   9.13372904 135.56652325  14.18378588         
        16.66273588  78.21171574]
on 3: [144.17204483  10.9018835   13.82580507 102.22842318  19.82908109         
         3.04445226  75.05501329]
on 4: Optimization terminated successfully.                                     
on 4:          Current function value: 0.000000                                 
on 4:          Iterations: 370                                                  
on 4:          Function evaluations: 826                                        
on 4: [149.99999164  10.32786534   6.40213565  96.08084805   7.27311798         
         2.08081226  71.3822858 ]
|████████████████████████████████████████| 5/5 [100%] in 2:52.4 (0.03/s)        
[array([120.99652309,  11.44496932,   4.06953278, 147.91960699,
        15.1965081 ,  11.2103605 ,  78.38435151]), array([141.91532763,

In [87]:
from maggeometry import n_rings, three_rings
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results2_nonun = []
results2_g_center = []
for x in results2_x:
#     innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2 = x
    obj_ring_config=[[*ringset_1_config, *x[0:3], 0], [*ringset_2_config, *x[3:7]]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    results2_nonun.append(obj1_nonuniformity(x))
    results2_g_center.append(center_field)

print(results2_nonun)
print(results2_g_center)

[3.216963615903097e-08, 1.074626373013218e-07, 3.839333373198834e-08, 6.431528704159967e-08, 8.612276255591128e-08]
[37.24220241207448, 10.00000000235886, 64.5103655481578, 38.596531967563976, 14.819278047638011]


In [91]:
bounds2 = ((2, 20), (2, 20), (2, 20), (2, 20), (1, 300))
def obj2_deviation(x):
    innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2 = x
    obj_ring_config = [[Br, mag_dir, mirror1_z, innerrad1, width1, thickness1, dist1],
                       [Br, mag_dir, mirror2_z, innerrad2, width2, thickness2, dist2]]
#     magnets = three_rings(Br, innerrad1, innerrad2, width1, thickness1, dist2)
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = abs(center_field - field_strength_target)
#     cost = np.log10(av_nonuniformity) + abs(center_field - field_strength_target)
    return cost

results2_x = []
results2_fun = []
method2 = 'Nelder-Mead'
options2 = {'disp':True, 'fatol': 1e-17,
#             'maxiter':0.5e3
           }
with alive_bar(len(st2g), force_tty=True) as bar:
    for s0 in st2g:
        def obj2_nonuniformity_fixed_rad(x):
            innerrad1 = s0[2]
            innerrad2 = s0[6]
            width1, thickness1, width2, thickness2, dist2 = x
            obj_ring_config = [[Br, mag_dir, mirror1_z, innerrad1, width1, thickness1, dist1],
                               [Br, mag_dir, mirror2_z, innerrad2, width2, thickness2, dist2]]
        #     magnets = three_rings(Br, innerrad1, innerrad2, width1, thickness1, dist2)
            magnets = n_rings(obj_ring_config)
            grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
#             cost = abs(center_field - field_strength_target)
#             cost = np.log10(av_nonuniformity) + abs(center_field - field_strength_target)
            cost = av_nonuniformity * abs(center_field - field_strength_target)
            return cost
        width1, thickness1 = s0[3:5]
        width2, thickness2, dist2 = s0[7:10]
        res2 = minimize(obj2_nonuniformity_fixed_rad, [width1, thickness1, width2, thickness2, dist2], method=method2, options=options2, bounds=bounds2)
        print(res2.x)
        results2_x.append(res2.x)
        results2_fun.append(res2.fun)
        bar()
        
print(results2_x)
print(results2_fun)

on 0: Optimization terminated successfully.                                     
on 0:          Current function value: 0.000001                                 
on 0:          Iterations: 473                                                  
on 0:          Function evaluations: 916                                        
on 0: [14.23834395 10.40789072  9.41809577 19.83653757 90.2714984 ]             
on 1: [18.21697422  2.00000017 19.12629692  2.01091452 89.29409477]             


on 1: /var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_50927/831269975.py:36: RuntimeWarning: Maximum number of function evaluations has been exceeded.
        res2 = minimize(obj2_nonuniformity_fixed_rad, [width1, thickness1, width2, thickness2, dist2], method=method2, options=options2, bounds=bounds2)


on 2: Optimization terminated successfully.                                     
on 2:          Current function value: 0.000000                                 
on 2:          Iterations: 235                                                  
on 2:          Function evaluations: 556                                        
on 2: [ 2.00007142  2.00000609  2.16584323  2.46662884 87.1183405 ]             
on 3: [19.97329813 20.         18.31835054 16.6228224  87.54306029]             
on 4: [19.62405213  2.17363672 16.55013714  2.         86.91847939]             
|████████████████████████████████████████| 5/5 [100%] in 1:59.5 (0.04/s)        
[array([14.23834395, 10.40789072,  9.41809577, 19.83653757, 90.2714984 ]), array([18.21697422,  2.00000017, 19.12629692,  2.01091452, 89.29409477]), array([ 2.00007142,  2.00000609,  2.16584323,  2.46662884, 87.1183405 ]), array([19.97329813, 20.        , 18.31835054, 16.6228224 , 87.54306029]), array([19.62405213,  2.17363672, 16.55013714,  2.      

In [93]:
from maggeometry import n_rings, three_rings
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results2_nonun = []
results2_g_center = []
for i in range(len(results2_x)):
    width1, thickness1, width2, thickness2, dist2 = results2_x[i]
    innerrad1 = st2g[i][2]
    innerrad2 = st2g[i][6]
#     innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2 = x
    obj_ring_config=[[*ringset_1_config, *x[0:3], 0], [*ringset_2_config, *x[3:7]]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    results2_nonun.append(obj1_nonuniformity(x))
    results2_g_center.append(center_field)

print(results2_nonun)
print(results2_g_center)

[0.0013279820224343661, 0.0013279820224343661, 0.0013279820224343661, 0.0013279820224343661, 0.0013279820224343661]
[10.0, 10.0, 10.0, 10.0, 10.0]


In [118]:
from maghelper import make_opt_res2_csv

r2_x = np.array(results2_x)
r2_x = np.insert(r_x, 3, 0, axis=1)
make_opt_res2_csv('results/2023_07_12_stage2opt_3rings_1090T_', results2_nonun, results2_g_center, r2_x, results2_fun)

Number of unique results: 17
    nonuniformity  center_field_gauss  r_1_innerrad  r_1_width  r_1_thickness  \
0    7.979775e-07                10.0    150.000000  19.996841       7.292153   
1    7.983628e-07                10.0    149.999982  19.900953       7.322989   
2    8.016443e-07                10.0    149.822903  20.000000       7.274975   
3    8.022097e-07                10.0    150.000000  20.000000       7.290980   
4    8.177803e-07                10.0    149.891733  20.000000       7.280847   
5    8.488074e-07                10.0    150.000000  19.999981       7.290366   
6    8.630953e-07                10.0    150.000000  16.174161       8.803304   
7    8.858650e-07                10.0    145.925076  20.000000       6.923850   
8    9.590014e-07                10.0    149.999293  20.000000       7.289532   
9    9.952847e-07                10.0    149.999937  19.998945       7.293496   
10   1.141181e-06                10.0    150.000000  19.999212       7.294070   

(array([[7.97977529e-07, 1.00000000e+01, 1.50000000e+02, 1.99968407e+01,
         7.29215266e+00, 2.70582713e+02, 1.13753801e-17],
        [7.98362759e-07, 1.00000000e+01, 1.49999982e+02, 1.99009531e+01,
         7.32298874e+00, 2.70506393e+02, 8.29633631e-18],
        [8.01644319e-07, 1.00000000e+01, 1.49822903e+02, 2.00000000e+01,
         7.27497546e+00, 2.70284201e+02, 4.02993802e-18],
        [8.02209738e-07, 1.00000000e+01, 1.50000000e+02, 2.00000000e+01,
         7.29097982e+00, 2.70580541e+02, 2.06982812e-17],
        [8.17780301e-07, 1.00000000e+01, 1.49891733e+02, 2.00000000e+01,
         7.28084732e+00, 2.70389346e+02, 7.23429476e-18],
        [8.48807400e-07, 1.00000000e+01, 1.50000000e+02, 1.99999808e+01,
         7.29036580e+00, 2.70562571e+02, 4.89276177e-18],
        [8.63095273e-07, 1.00000000e+01, 1.50000000e+02, 1.61741609e+01,
         8.80330421e+00, 2.67469765e+02, 1.69491412e-17],
        [8.85864976e-07, 1.00000000e+01, 1.45925076e+02, 2.00000000e+01,
         6